In [1]:
# Dependencies and setup
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from splinter import Browser

# Mars Facts

In [2]:
# URL: https://space-facts.com/mars/

# Visit the Mars Facts webpage here and use Pandas to scrape the 
# table containing facts about the planet including Diameter, Mass, etc.

# Use Pandas to convert the data to a HTML table string.

In [3]:
# Store the url
url = 'https://space-facts.com/mars'

# Scrape tabular data from the website
tables = pd.read_html(url)

In [4]:
# Store the desired table
fact_table = tables[0]

In [5]:
# Clean up table dataframe
# Rename columns
fact_table.columns = ['Attribute', 'Value']
# Set 'Attribute' column as the index
fact_table.set_index('Attribute', inplace=True)

In [6]:
# Save html table as a file
fact_table.to_html('fact-table.html')